In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-f574g314
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-f574g314
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=6233d256c560415a76cd62a5735071772c335d7421c7b58a3e2faf3b806eaa55
  Stored in directory: /tmp/pip-ephem-wheel-cache-z9qg31sb/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [ ]:
    import random
    #def generate(n):
    n=10;
    s=str(n)+"\n";
    for i in range(1,n):
      for i in range(1,n+1):
            s += str(random.randint(0,10))+" ";  
      s+="\n"; 
    file1 = open("/content/data.txt","w")#write mode 
    file1.write(s) 
    file1.close()   
    #return s;

  

In [ ]:
%%cu
#include <cuda_runtime.h>
#include <stdio.h>
#include <device_launch_parameters.h>
#include <stdlib.h>
#include <time.h>
#include <sys/time.h>
#include <unistd.h>

#define THREADS_PER_BLOCK 16

void save_to_file(double *AB, const int a_size) {
    FILE *f = fopen("/content/out.txt", "w+");

    fprintf(f, "%d\n", a_size);

    for(int i = 0; i < a_size*(a_size + 1); i++) {
        if((i + 1) % (a_size+1) == 0) {
            fprintf(f, "\n");
            continue;
        }
        fprintf(f, "%lf ", AB[i]);
    }

    for(int i = 0; i < a_size; i++)
        fprintf(f, "%lf ", AB[a_size + i*(a_size + 1)]);
    fprintf(f, "\n");

    fclose(f);
}

double *malloc_matrix(const int a, const int b) {
    return (double*)malloc(sizeof(double *)*a*b);
}

double *load_from_file(int *a_size, char *name) {
    FILE *f = fopen(name, "r");

    int size;
    fscanf(f, "%d", &size);
    double *matrix_ab = malloc_matrix(size, size + 1);

    for(int i = 0; i < size*(size + 1); i++) {
        if((i+1) % (size + 1) == 0)
            continue;
        fscanf(f, "%lf", &matrix_ab[i]);
    }

    for(int i = 0; i < size; i++) {
        fscanf(f, "%lf", &matrix_ab[size + i*(size + 1)]);
    }

    fclose(f);

    *a_size = size;

    return matrix_ab;
}


void print_matrix(double *matrix, const int a, const int b) {
    for(int i = 0; i < a*b; i++) {
        printf("%lf\t", matrix[i]);
        if((i+1) % b == 0)
            printf("\n");
    }
}

void print_output(double *AB, const int a_size) {
    printf("%d\n", a_size);

    for(int i = 0; i < a_size*(a_size + 1); i++) {
        if((i + 1) % (a_size+1) == 0) {
            printf("\n");
            continue;
        }
        printf("%lf\t", AB[i]);
    }

    for(int i = 0; i < a_size; i++)
        printf("%lf\t", AB[a_size + i*(a_size + 1)]);
    printf("\n");
}

int load_size() {
    int size;
    scanf("%d", &size);
    return size;
}

void load_ab_matrix(double *a, const int size) {
    for(int i = 0; i < size*(size + 1); i++) {
        if((i+1) % (size + 1) == 0)
            continue;
        scanf("%lf", &a[i]);
    }

    for(int i = 0; i < size; i++) {
        scanf("%lf", &a[size + i*(size + 1)]);
    }
}

double  *load_input(int *size) {
    *size = load_size();
    double *matrix_ab = malloc_matrix(*size, *size + 1);
    load_ab_matrix(matrix_ab, *size);
    return matrix_ab;
}


__global__ void replace_zero_gpu(double *AB, int rows, int columns, int column) {
    if(fabs(AB[column*columns + column]) <= 1e-4) {
        int row = column;
        for(; row < rows; row++) {
            if(fabs(AB[row*columns + column]) > 1e-4)
                break;
        }
        int threadId = blockDim.x*blockIdx.x + threadIdx.x;
        if(threadId + column >= columns)
            return;

        int zero = column*columns + column + threadId;
        int chosen = row*columns + column + threadId;
        AB[zero] += AB[chosen];
    }
}


__global__ void column_elimination_gpu(double *AB, int rows, int columns, int column) {
    int threadId = blockDim.x*blockIdx.x + threadIdx.x;
    if(threadId >= (rows - 1 - column)*(columns - column))
        return;

    int el_row = column + threadId/(columns - column) + 1;
    int el_col = column + threadId%(columns - column);
    int el = el_col + el_row*columns;
    int upper_el = el_col + column*columns;

    int main_el = column + column*columns;
    int main2_el = column + el_row*columns;
    double f = AB[main2_el]/AB[main_el];

    AB[el] -= f*AB[upper_el];
}

__global__ void multiple_column(double *AB, int rows, int columns, int row) {
    int threadId = threadIdx.x;
    AB[(threadId * columns) + row] *= AB[columns*(row + 1) - 1];
}

__global__ void reverse_row_elimination(double *AB, int rows, int columns, int row) {
    int threadId = threadIdx.x;
    int cols = columns - 2 - row;

    int start_index = row*columns + row + 1;

    int j = cols%2;
    for(int i = cols/2; i > 0; i/=2) {
        if(threadId >= i)
            return;

        AB[start_index + threadId] += (AB[start_index + threadId + i + j]);
        AB[start_index + threadId + i + j] = 0;
        if(j == 1)
            i++;
        j = i%2;
        __syncthreads();
    }

    int x_el = (row + 1)*columns - 1;
    int diag_el = row*columns + row;

    if(diag_el + 1 != x_el) {
        AB[x_el] -= AB[diag_el + 1];
        AB[diag_el + 1] = 0.0;
    }

    AB[x_el] /= AB[diag_el];
    AB[diag_el] = 1.0;
}

__global__ void sum_row(double *AB, int rows, int columns, int row) {
    int threadId = threadIdx.x;

    int j = columns%2;
    for(int i = columns/2; i > 0; i/=2) {
        if(threadId >= i)
            return;

        AB[threadId] += AB[threadId + i + j];
        __syncthreads();
        if(j == 1)
            i++;
        j = i%2;
    }
}


void start_gaussian_elimination_gpu(double *AB, int rows, int cols) {
    double *AB_gpu;

    cudaMalloc(&AB_gpu, sizeof(double)*rows*cols);
    cudaMemcpy(AB_gpu, (void*)AB, sizeof(double)*rows*cols, cudaMemcpyHostToDevice);

    int block_size;

    for(int column = 0; column < cols - 1; column++) {
        block_size = (cols - column - 1)/THREADS_PER_BLOCK + 1;
        replace_zero_gpu<<<block_size, THREADS_PER_BLOCK>>>(AB_gpu, rows, cols, column);
        cudaThreadSynchronize();

        block_size = ((rows - column)*(cols - column) - 1)/THREADS_PER_BLOCK + 1;
        column_elimination_gpu<<<block_size, THREADS_PER_BLOCK>>>(AB_gpu, rows, cols, column);
        cudaThreadSynchronize();
    }

    for(int row = rows - 1; row >= 0; row--) {
        reverse_row_elimination<<<1, cols>>>(AB_gpu, rows, cols, row);
        multiple_column<<<1, row>>>(AB_gpu, rows, cols, row);

        cudaThreadSynchronize();
    }

    cudaMemcpy(AB, (void*)AB_gpu, sizeof(double)*rows*cols, cudaMemcpyDeviceToHost);

    cudaFree(AB_gpu);
}


int main(int argc, char ** argv) {
    int size;
    double *AB = load_from_file(&size, "/content/data.txt");

    print_output(AB, size);

       
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);
    
    
    start_gaussian_elimination_gpu(AB, size, size + 1);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);
    
     
    printf("\n\n");

    print_output(AB, size);

    save_to_file(AB, size);

    printf("\nTime Taken:  %f sec",(milliseconds/1000));

    return 0;
}

10
9.000000	7.000000	4.000000	3.000000	2.000000	7.000000	0.000000	3.000000	4.000000	4.000000	
1.000000	10.000000	3.000000	10.000000	0.000000	6.000000	9.000000	0.000000	6.000000	5.000000	
2.000000	1.000000	0.000000	10.000000	5.000000	9.000000	2.000000	0.000000	6.000000	9.000000	
4.000000	4.000000	10.000000	7.000000	9.000000	10.000000	0.000000	10.000000	3.000000	4.000000	
6.000000	10.000000	6.000000	1.000000	0.000000	7.000000	10.000000	9.000000	7.000000	7.000000	
4.000000	7.000000	0.000000	8.000000	5.000000	2.000000	9.000000	6.000000	1.000000	7.000000	
7.000000	10.000000	10.000000	9.000000	1.000000	4.000000	7.000000	10.000000	6.000000	8.000000	
1.000000	4.000000	2.000000	3.000000	0.000000	2.000000	6.000000	2.000000	6.000000	9.000000	
3.000000	1.000000	6.000000	3.000000	10.000000	1.000000	3.000000	1.000000	10.000000	3.000000	
0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	
0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.